In [1]:
import pandas as pd
import re

In [2]:
df = pd.read_csv('diagnosis.csv', keep_default_na=False)
new_df = pd.DataFrame(columns=list(df))
df.head()

,patient_id,encounter_id,encounter_date,patient_sex,patient_age,smoker_status,tag_name,tag_id,tag_time,tag_type1,tag_type2,tag_child_text
0,293,3,2077-07-30,female,71,current,DIABETES,DOC3,before DCT,NA,NA,A1C: HA1C of 8.0.; A1C: HA1C of 8.0
1,293,3,2077-07-30,female,71,current,HYPERTENSION,DOC4,during DCT,NA,NA,mention: htn; mention: HTN
2,293,3,2077-07-30,female,71,current,DIABETES,DOC5,before DCT,NA,NA,mention: DM
3,293,3,2077-07-30,female,71,current,DIABETES,DOC7,during DCT,NA,NA,mention: DM
4,293,3,2077-07-30,female,71,current,HYPERTENSION,DOC11,after DCT,NA,NA,mention: htn; mention: HTN


In [3]:
next_patient = 293
next_encounter = 3
next_tag_name = "DIABETES"

this_group = df[(df.patient_id == next_patient) & 
                (df.encounter_id == next_encounter) & 
                (df.tag_name == next_tag_name)]

In [4]:
wanted_headers = [x for x in list(df) if x not in ['tag_id', 'tag_type1', 'tag_type2']]

new_df = pd.DataFrame(columns=wanted_headers)
new_df

,patient_id,encounter_id,encounter_date,patient_sex,patient_age,smoker_status,tag_name,tag_time,tag_child_text


In [5]:
next_row = 0
for next_patient in list(set(df.patient_id)):
    patient_df = df[df.patient_id == next_patient]
    for next_encounter in list(set(patient_df.encounter_id)):
        encounter_df = patient_df[patient_df.encounter_id == next_encounter]
        for next_tag in list(set(encounter_df.tag_name)):
            this_group = encounter_df[encounter_df.tag_name == next_tag]
            
            next_row += 1
            for col in list(new_df):
                new_df.loc[next_row, col] = "; ".join([str(x) for x in set(this_group[col])])
    
new_df.head()

,patient_id,encounter_id,encounter_date,patient_sex,patient_age,smoker_status,tag_name,tag_time,tag_child_text
1,220,1,2067-05-03,female,55,past,HYPERLIPIDEMIA,during DCT; after DCT; before DCT,mention: hyperlipidemia
2,220,1,2067-05-03,female,55,past,HYPERTENSION,during DCT; after DCT; before DCT,mention: Hypertension; mention: HTN; high bp: ...
3,220,1,2067-05-03,female,55,past,CAD,during DCT; after DCT; before DCT,mention: CAD; mention: PLAVIX; mention: corona...
4,220,2,2068-12-05,female,37,past,HYPERLIPIDEMIA,during DCT; after DCT; before DCT,mention: Hyperlipidemia
5,220,2,2068-12-05,female,37,past,HYPERTENSION,during DCT; after DCT; before DCT,mention: Hypertension; mention: HTN


In [6]:
def does_have_number(a_string):
    for letter in a_string:
        try:
            int(letter)
            return True
        except ValueError:
            pass
    return False

does_have_number('asdf.121g')

True

In [7]:
new_df['tag_child_nums'] = 'NA'

for row in xrange(1, new_df.shape[0]):
    text_list = new_df.loc[row, 'tag_child_text'].split('; ')

    num_text = [text for text in text_list if does_have_number(text)]
    if len(num_text) == 0:
        num_text = ['NA']
    new_df.loc[row, 'tag_child_nums'] = '; '.join(num_text)
new_df.head()

,patient_id,encounter_id,encounter_date,patient_sex,patient_age,smoker_status,tag_name,tag_time,tag_child_text,tag_child_nums
1,220,1,2067-05-03,female,55,past,HYPERLIPIDEMIA,during DCT; after DCT; before DCT,mention: hyperlipidemia,NA
2,220,1,2067-05-03,female,55,past,HYPERTENSION,during DCT; after DCT; before DCT,mention: Hypertension; mention: HTN; high bp: ...,high bp: 170/80; high bp: BP 170/80; high bp: ...
3,220,1,2067-05-03,female,55,past,CAD,during DCT; after DCT; before DCT,mention: CAD; mention: PLAVIX; mention: corona...,event: s/p ant SEMI + stent LAD 2/67- Dr Oakley
4,220,2,2068-12-05,female,37,past,HYPERLIPIDEMIA,during DCT; after DCT; before DCT,mention: Hyperlipidemia,NA
5,220,2,2068-12-05,female,37,past,HYPERTENSION,during DCT; after DCT; before DCT,mention: Hypertension; mention: HTN,NA


In [8]:
def remove_punctuation(a_string):
    for char in [':', ';','=','-', ')', '(', '%']:
        a_string = a_string.replace(char, '')
    return a_string.lower().strip()

def get_bp(a_string):
    a_string = remove_punctuation(a_string)
    bp_list = list()  
    
    for i in re.finditer('/', a_string):
        new_text = a_string[i.start()-3:i.end()+3].strip()
        new_list = new_text.split('/')
        if len(new_list) == 2:
            try:
                the_sys = float(new_list[0])
                the_dia = float(new_list[1])
                if (the_sys > 30) and (the_dia > 10): #these number picked as clear exmaples of too low
                    bp_list.append({'sys': the_sys, 'dia': the_dia})
            except ValueError:
                pass
    
    if len(bp_list) == 1:
        return bp_list[0]
    elif len(bp_list) == 0:
        return None
    else:
        sys_avg = 0
        dia_avg = 0
        for each in bp_list:
            sys_avg += each['sys']
            dia_avg += each['dia']
        sys_avg = sys_avg / len(bp_list)
        dia_avg = dia_avg / len(bp_list)
        return({'sys': round(sys_avg, 2), 'dia': round(dia_avg, 2)})
        
text = '150/70 repeat 145/80; 145/80; 150/70; '
get_bp(text)

{'dia': 75.0, 'sys': 147.5}

In [9]:
def get_chol(a_string):
    chol_dict = {}
    child_list = a_string.split(';')
    
    for chol_type in ['high LDL', 'high chol']:
        num_list = list()
        for child in child_list:
            if chol_type in child:
                word_list = child.split(' ')
                word_list = [remove_punctuation(x) for x in word_list]

                for word in word_list:
                    if len(word) == 3:
                        try:
                            int(word)
                            num_list.append(int(word))
                        except ValueError:
                            pass
            try:
                chol_value = sum(num_list) / len(num_list)
            except ZeroDivisionError: 
                chol_value = 'NA'
            chol_dict[chol_type] = chol_value
    if len (chol_dict) == 0:
        return None
    else:
        return chol_dict

get_chol('high chol.: total cholesterol was 294; high chol.: his total cholesterol was 294; high LDL: his LDL was 209; high LDL: LDL was 209')

{'high LDL': 209, 'high chol': 294}

In [10]:
def get_sugar(a_string):
    sugar_dict = {}
    child_list = a_string.split(';')
    
    for measure_type in ['A1C', 'glucose']:
        num_list = list()
        for child in child_list:
            if measure_type in child:
                word_list = child.split(' ')
                word_list = [remove_punctuation(x) for x in word_list]

                for word in word_list:
                    if len(word) == 3 or (measure_type == 'A1C' and len(word) == 4):
                        try:
                            float(word)
                            num_list.append(float(word))
                        except ValueError:
                            pass
            try:
                sugar_value = sum(num_list) / len(num_list)
            except ZeroDivisionError: 
                sugar_value = 'NA'
            sugar_dict[measure_type] = sugar_value
    if len(sugar_dict) == 0:
        return None
    else:
        return sugar_dict



get_sugar('A1C: hgba1c 7.6 %; A1C: hgba1c 7.6 % (from 7.8%); A1C: hgba1c 7.6; A1C: 7.8%); glucose: FS in the AM ~ 90-180; glucose: 180')

{'A1C': 7.68, 'glucose': 180.0}

In [11]:
new_df['measure_1'] = 'NA'
new_df['measure_2'] = 'NA'

for row in xrange(1, new_df.shape[0]):
    if new_df.loc[row, 'tag_name'] == "HYPERTENSION":
        bp = get_bp(new_df.loc[row, 'tag_child_nums'])
        if bp is not None:
            new_df.loc[row, 'measure_1'] = bp['sys']
            new_df.loc[row, 'measure_2'] = bp['dia']
            
    if new_df.loc[row, 'tag_name'] == "HYPERLIPIDEMIA":
        chol = get_chol(new_df.loc[row, 'tag_child_nums'])
        if chol is not None:
            new_df.loc[row, 'measure_1'] = chol['high LDL']
            new_df.loc[row, 'measure_2'] = chol['high chol']
    
    if new_df.loc[row, 'tag_name'] == "DIABETES":
        sugar = get_sugar(new_df.loc[row, 'tag_child_nums'])
        if bp is not None:
            new_df.loc[row, 'measure_1'] = sugar['A1C']
            new_df.loc[row, 'measure_2'] = sugar['glucose']
            
new_df.head()

,patient_id,encounter_id,encounter_date,patient_sex,patient_age,smoker_status,tag_name,tag_time,tag_child_text,tag_child_nums,measure_1,measure_2
1,220,1,2067-05-03,female,55,past,HYPERLIPIDEMIA,during DCT; after DCT; before DCT,mention: hyperlipidemia,NA,NA,NA
2,220,1,2067-05-03,female,55,past,HYPERTENSION,during DCT; after DCT; before DCT,mention: Hypertension; mention: HTN; high bp: ...,high bp: 170/80; high bp: BP 170/80; high bp: ...,157.14,77.14
3,220,1,2067-05-03,female,55,past,CAD,during DCT; after DCT; before DCT,mention: CAD; mention: PLAVIX; mention: corona...,event: s/p ant SEMI + stent LAD 2/67- Dr Oakley,NA,NA
4,220,2,2068-12-05,female,37,past,HYPERLIPIDEMIA,during DCT; after DCT; before DCT,mention: Hyperlipidemia,NA,NA,NA
5,220,2,2068-12-05,female,37,past,HYPERTENSION,during DCT; after DCT; before DCT,mention: Hypertension; mention: HTN,NA,NA,NA


In [12]:
new_df['condition'] = new_df['tag_name']
shrunk_df = new_df.drop(['tag_child_text', 'tag_child_nums', 'tag_time', 'tag_name'], 1)


In [13]:
shrunk_df.to_csv('diagnosis_simplified.csv', index_label=False)